# Building an Interactive Q&A Retriever

This tutorial delves into the creation of a robust question and answer (Q&A) retrieval system using cutting-edge libraries in a notebook environment. Here's a sneak peek into the journey:

1. **Library Installation:** Explore the installation of essential Python packages like `langchain`, `faiss-cpu`, `sentence_transformers`, and more, laying the foundation for building the Q&A system.

2. **API Key Setup:** Learn how to retrieve and utilize user API keys, crucial for accessing external services like Together AI.

3. **Q&A Retriever Setup:** Dive into setting up the Q&A retrieval system, utilizing advanced techniques such as text embeddings, vector databases, and language models.

4. **Data Preparation:** Understand the process of loading and splitting text data, leveraging techniques like web scraping and text chunking for efficient indexing.

5. **Example Data with Metadata:** Explore the creation of example datasets with attached metadata, essential for training and testing the Q&A system.

6. **Retriever Construction:** Witness the construction of a powerful retriever, capable of searching and retrieving relevant documents based on user queries.

7. **Language Model Integration:** Integrate advanced language models like Together AI's Open Hermes to generate accurate answers based on retrieved context.

8. **Pipeline Creation:** Learn how to create a seamless pipeline or "chain" by combining retrievers, prompt templates, language models, and output parsers, enabling the system to handle user questions effectively.

9. **Chain Invocation:** Experience the magic as the Q&A system springs to life, effortlessly answering sample questions based on retrieved context and language model predictions.

In [4]:
!pip -q install langchain langchain-core langchain-community faiss-cpu tiktoken langchain-together sentence_transformers gradio -qqq

In [5]:
from google.colab import userdata

import textwrap
import os

os.environ["TOGETHER_API_KEY"] = userdata.get('TOGETHER_API_KEY')

## Q&A-Retreiver
 This tutorial provides an overview of creating a question and answer (Q&A) retrieval system using various libraries in a notebook setting. Here are the main components and their functions:

1. Installing required libraries: The code installs several Python packages, including `langchain`, `langchain-core`, `langchain-community`, `faiss-cpu`, `tiktoken`, `sentence_transformers`, and `Gradio`. These libraries are used to build the Q&A system.

2. Loading user API key: The script retrieves an API key from the notebook's user data, which is likely needed for some external services like Together AI.

3. Setting up the Q&A retriever: The tutorial demonstrates creating a question-answering system using different embedding models and vector databases. Here, `HuggingFaceEmbeddings` from the `langchain-community` library is used with the `intfloat/multilingual-e5-small` model for text embeddings. The embedded documents are stored in a FAISS (FAST AI Similarity Search) vector store.

4. Loading and splitting data: The code loads text content from either a web page or a local file using `WebBaseLoader` or `TextLoader`. The loaded text is then split into smaller chunks using the `RecursiveCharacterTextSplitter` for efficient indexing.

5. Creating an example data set with metadata: The split documents are used to create a vector store, which serves as the searchable database for the Q&A system.

6. Building a retriever: A retriever is created from the FAISS vector store, allowing it to search and retrieve relevant documents based on user queries. In this case, the `search_kwargs` parameter specifies that three most similar documents should be returned for each query.

7. Integrating a language model: The tutorial uses Together AI's Open Hermes model (`teknium/OpenHermes-2p5-Mistral-7B`) to generate answers based on the retrieved context. The `Together` class from the `langchain-together` library is used for this purpose.

8. Creating a chain: A pipeline, or "chain," is created by combining the retriever, prompt template, language model, and output parser. This chain allows the system to take a question as input and return an answer based on the retrieved context.

9. Invoking the chain: Finally, the chain is invoked with a sample question ("How did they start Instagram?") to demonstrate how the Q&A system works. The answer is generated by combining the relevant documents retrieved from the vector store and using the language model to generate an appropriate response.


In [6]:
#from langchain.embeddings.openai import OpenAIEmbeddings
#from langchain_together.embeddings import TogetherEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

#embeddings = OpenAIEmbeddings()
#embeddings = TogetherEmbeddings(model="togethercomputer/m2-bert-80M-2k-retrieval")
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-small")

from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import TextLoader

In [9]:
import bs4

In [10]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://lexfridman.com/sam-altman-2-transcript/")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

## Example data with metadata attached

In [11]:
# get the wines in the store
vectorstore = FAISS.from_documents(splits, embeddings)

## Creating our retriever

In [12]:
from langchain_together import Together

In [19]:
llm = Together(
    model="teknium/OpenHermes-2p5-Mistral-7B",
    temperature=0.3,
    max_tokens=512,
    top_k=50,
    # together_api_key="..."
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [15]:
retriever.get_relevant_documents('What do they say about Elon?')

[Document(page_content='Elon Musk lawsuit\n\nLex Fridman\n(00:24:39) \nOur mutual friend Elon sued OpenAI. What to you is the essence of what he’s criticizing? To what degree does he have a point? To what degree is he wrong?\n\n\nSam Altman\n(00:24:52) \nI don’t know what it’s really about. We started off just thinking we were going to be a research lab and having no idea about how this technology was going to go. Because it was only seven or eight years ago, it’s hard to go back and really remember what it was like then, but this is before language models were a big deal. This was before we had any idea about an API or selling access to a chatbot. It was before we had any idea we were going to productize at all. So we’re like, “We’re just going to try to do research and we don’t really know what we’re going to do with that.” I think with many fundamentally new things, you start fumbling through the dark and you make some assumptions, most of which turned out to be wrong.', metadata={'

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [17]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
chain.invoke("What do they say about Elon?")

'\nAnswer: They say that Elon is a friend and a beautiful human being, and one of the most important humans ever. They also mention that Elon is a great leader and that all of them should be rooting for him to step up as a leader through the next phase.'